### Este archivo tiene como proposito extraer la informacion de posts de paginas de Facebook. Fue utilizado para extraer posts de paginas de periodicos dominicanos.

In [3]:
# import some Python dependencies

import urllib.request as urllib2
import json
import datetime
import csv
import time

In [4]:
# Este es mi app id y app secret, cambienlo al suyo pls.
app_id = "175344463016958"
app_secret = "3ab6d67e8ab1f74095b8e4d9e1cff37e" 

access_token = app_id + "|" + app_secret

In [5]:
page_id = 'DiarioLibre' # Reemplazan este ID por el de la página que desean.

In [16]:
def request_until_succeed(url):
    """
    Intenta abrir un url "a la mala", reintentando un request hacia la página hasta que sea exitoso. 
    Si la página no existe, será infinito.
    params:
        url- string: URL de la página que se va a abrir.
    
    outputs:
        response.read()- string: Documento HTML (o json) con los datos de la página.
    """
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except:
            time.sleep(5)
            
            print ("Error for URL %s: %s" % (url, datetime.datetime.now()))

    return response.read()

In [92]:
import pandas as pd
def testFacebookPageFeedData(page_id, access_token):
    """
    Envía un request a la API de grafo de Facebook, para la sección de news feeds de una página. Luego, obtiene todos los posts,
    su contenido, y total de reacciones.
    
    params:
        page_id- string: ID de la página de la cual se extraerán los datos.
        access_token: string: Llave del API de Facebook.
        
    outputs:
        data: json dict: JSON con los resultados de los datos extraídos.
    """
    # construct the URL string
    base = "https://graph.facebook.com/v2.10"
    node = "/" + page_id + "/feed" # changed
    parameters = "/?fields=message,created_time,reactions.type(LOVE).limit(0).summary(total_count).as(reactions_love),reactions.type(WOW).limit(0).summary(total_count).as(reactions_wow),reactions.type(HAHA).limit(0).summary(total_count).as(reactions_haha),reactions.type(ANGRY).limit(0).summary(total_count).as(reactions_angry),reactions.type(SAD).limit(0).summary(total_count).as(reactions_sad),reactions.type(LIKE).limit(0).summary(total_count).as(reactions_like)&limit={}&access_token={}".format(100, access_token) # changed
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    return data

In [ ]:

def Get_News(limit = 10):
    """
    Extrae las reacciones, fecha de creación y contenido de posts de una página de Facebook. Para el primer intento, 
    llama al API directamente. Para los intentos siguientes, llama al parametro "Next" de los resultados del API.
    params:
        limit- int: Cuantas veces se llama al API de FB para extraer noticias.
    
    output:
        result_df-Pandas DataFrame: Tabla con los valores extraídos por cada post de Facebook.
    """
    result = {}
    nex = None
    for  i in range(limit):
        range_dates = []
        range_messages = []
        range_angry = []
        range_haha = []
        range_like = []
        range_love = []
        range_sad = []
        range_wow = []
        range_ids=  []
        if i == 0:
            data = testFacebookPageFeedData(page_id,access_token)
            nex = data['paging']['next']
            for d in data['data']:
                range_dates.append(d['created_time'])
                range_messages.append(d['message'])
                range_angry.append(d['reactions_angry']['summary']['total_count'])
                range_haha.append(d['reactions_haha']['summary']['total_count'])
                range_like.append(d['reactions_like']['summary']['total_count'])
                range_love.append(d['reactions_love']['summary']['total_count'])
                range_sad.append(d['reactions_sad']['summary']['total_count'])
                range_wow.append(d['reactions_wow']['summary']['total_count'])
                range_ids.append(d['id'])
            result['dates'] = range_dates
            result['messages'] = range_messages
            result['angry'] = range_angry
            result['haha'] = range_haha
            result['like'] = range_like
            result['love'] = range_love
            result['sad'] = range_sad
            result['wow'] = range_wow
            result['id'] = range_ids
        
        else:
            data = json.loads(request_until_succeed(nex))
            try:
                nex = data['paging']['next']
            except:
                break
            for d in data['data']:
                try:
                    range_messages.append(d['message'])
                    range_dates.append(d['created_time'])
                    range_angry.append(d['reactions_angry']['summary']['total_count'])
                    range_haha.append(d['reactions_haha']['summary']['total_count'])
                    range_like.append(d['reactions_like']['summary']['total_count'])
                    range_love.append(d['reactions_love']['summary']['total_count'])
                    range_sad.append(d['reactions_sad']['summary']['total_count'])
                    range_wow.append(d['reactions_wow']['summary']['total_count'])
                    range_ids.append(d['id'])
                    
                except:
                    print(d)
            result['dates'].extend(range_dates)
            result['messages'].extend(range_messages)
            result['angry'].extend(range_angry)
            result['haha'].extend(range_haha)
            result['like'].extend(range_like)
            result['love'].extend(range_love)
            result['sad'].extend(range_sad)
            result['wow'].extend(range_wow)
            result['id'].extend(range_ids)
            
    
    result_df = pd.DataFrame(result)
    return result_df

In [93]:
news = Get_News(1000)
news.shape

{'created_time': '2017-10-01T10:45:01+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 30}}, 'id': '172728406899_10155506642331900'}
{'created_time': '2013-12-31T10:50:00+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 1}}, 'id': '172728406899_10152114476346900'}
{'created_time': '2013-12-18T09:50:01+0000', 'reactions_love': {'data': [],

{'created_time': '2012-06-05T19:13:42+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 61}}, 'id': '172728406899_390848364284257'}
{'created_time': '2012-06-04T19:25:28+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 104}}, 'id': '172728406899_454114681284504'}
{'created_time': '2012-05-29T16:44:13+0000', 'reactions_love': {'data': [], '

{'created_time': '2012-05-04T14:59:34+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 43}}, 'id': '172728406899_143585682432975'}
{'created_time': '2012-05-03T16:20:03+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 77}}, 'id': '172728406899_234367086663136'}
{'created_time': '2012-04-25T16:57:52+0000', 'reactions_love': {'data': [], 's

{'created_time': '2010-10-07T19:24:17+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 13}}, 'id': '172728406899_119551401436789'}
{'created_time': '2010-10-05T19:41:46+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 21}}, 'id': '172728406899_155666081135025'}
{'created_time': '2010-10-05T16:46:41+0000', 'reactions_love': {'data': [], 's

{'created_time': '2010-07-29T13:17:56+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 30}}, 'id': '172728406899_141014035919428'}
{'created_time': '2010-07-28T15:44:11+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 24}}, 'id': '172728406899_141119462584430'}
{'created_time': '2010-07-28T13:09:47+0000', 'reactions_love': {'data': [], 's

{'created_time': '2010-06-09T18:14:57+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 9}}, 'id': '172728406899_133953493286569'}
{'created_time': '2010-06-09T13:34:21+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 23}}, 'id': '172728406899_130344790325350'}
{'created_time': '2010-06-02T17:57:52+0000', 'reactions_love': {'data': [], 'su

{'created_time': '2009-11-12T15:21:12+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 0}}, 'id': '172728406899_200254630188'}
{'created_time': '2009-11-10T19:03:42+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 3}}, 'id': '172728406899_176538059903'}
{'created_time': '2009-11-10T16:40:33+0000', 'reactions_love': {'data': [], 'summary':

{'created_time': '2009-11-03T19:36:59+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 6}}, 'id': '172728406899_189324525968'}
{'created_time': '2009-11-03T18:41:48+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 2}}, 'id': '172728406899_172515291646'}
{'created_time': '2009-11-03T15:51:12+0000', 'reactions_love': {'data': [], 'summary':

{'created_time': '2009-10-21T14:36:33+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 31}}, 'id': '172728406899_160832848157'}
{'created_time': '2009-10-21T12:25:43+0000', 'reactions_love': {'data': [], 'summary': {'total_count': 0}}, 'reactions_wow': {'data': [], 'summary': {'total_count': 0}}, 'reactions_haha': {'data': [], 'summary': {'total_count': 0}}, 'reactions_angry': {'data': [], 'summary': {'total_count': 0}}, 'reactions_sad': {'data': [], 'summary': {'total_count': 0}}, 'reactions_like': {'data': [], 'summary': {'total_count': 16}}, 'id': '172728406899_158486673381'}
{'created_time': '2009-10-20T18:52:08+0000', 'reactions_love': {'data': [], 'summary

(5551, 9)

In [94]:
news.shape

(5551, 9)

In [95]:
news.tail()

,angry,dates,haha,id,like,love,messages,sad,wow
5546,0,2009-10-12T13:29:33+0000,0,172728406899_169814206096,16,0,Felicitamos a quienes están de cumpleaños hoy....,0,0
5547,0,2009-10-12T13:25:42+0000,0,172728406899_181448350637,6,0,http://bit.ly/3vAhhf,0,0
5548,0,2009-10-12T12:17:36+0000,0,172728406899_176656473477,4,0,http://bit.ly/4yPwQq,0,0
5549,0,2009-10-12T12:04:09+0000,0,172728406899_157285692342,20,0,Boquechivo de hoy: La playa ta jodona,0,0
5550,0,2009-10-12T11:57:08+0000,0,172728406899_150937058662,20,0,Buenos días lectores ¿cómo los trató el fin de...,0,0


### Luego de haber extraído las noticias, se cambian las fechas a un tipo de dato fecha, y se extrae el año y mes.

In [176]:
news.dates=pd.to_datetime(news.dates)
news['month'] = news.dates.dt.month
news['year'] = news.dates.dt.year
news.loc[news.dates.dt.date.astype(str) == '2017-10-06']

,angry,dates,haha,id,like,love,messages,sad,wow,month,year
34,0,2017-10-06 23:40:00,0,172728406899_10155521783921900,16,0,Exigen acelerar el proceso de naturalización e...,0,0,10,2017
35,0,2017-10-06 23:20:00,8,172728406899_10155521780611900,839,4,El huracán María devastó la Isla de los Monos ...,430,162,10,2017
36,0,2017-10-06 23:00:00,0,172728406899_10155521777626900,46,0,Las grandes empresas que amenazan sacar su sed...,0,3,10,2017
37,0,2017-10-06 22:40:00,0,172728406899_10155521775706900,26,0,Google y el precio de las noticias pagadas htt...,0,0,10,2017
38,0,2017-10-06 22:20:00,0,172728406899_10155521767376900,61,0,La Policía pone en duda que el tirador de Las ...,0,6,10,2017
39,0,2017-10-06 22:00:00,1,172728406899_10155521765831900,32,0,#Argentarium: ¿Y si ahorro un peso diario? htt...,0,0,10,2017
40,2,2017-10-06 21:45:00,32,172728406899_10155521755946900,319,6,Boricuas le piden al vicepresidente de Estados...,4,10,10,2017
41,0,2017-10-06 21:30:00,0,172728406899_10155521753821900,7,0,"#FUNDEU: Bulevar, mejor que boulevard https://...",0,1,10,2017
42,2,2017-10-06 21:15:00,0,172728406899_10155521749466900,15,3,Deuda Sector Público No Financiero creció US$2...,0,0,10,2017
43,0,2017-10-06 21:00:00,0,172728406899_10155521969521900,26,0,"En #GastroClubDL: Gastritis, ¿existe realmente...",0,0,10,2017


### Se mantienen solo las noticias con urls.

In [85]:
valid_news = news.loc[((news.messages.str.contains('http')) | (news.messages.str.contains('ht'))) & (~news.messages.str.contains('twitter.com'))]
valid_news.shape

(2445, 8)

In [144]:
date_news = valid_news.copy()
date_news.dates = pd.to_datetime(date_news.dates)
date_news.head()

,angry,dates,haha,like,love,messages,sad,wow
0,0,2017-10-07 16:26:00,0,4,0,#GuíadelDinero: ¿Cómo hacer el ejercicio de or...,0,0
1,1,2017-10-07 15:51:32,1,457,1,Fallece paracaidista dominicano tras accidente...,484,78
2,0,2017-10-07 15:42:27,2,33,0,Falleció en la madrugada jurista Domingo Porfi...,14,9
3,0,2017-10-07 15:35:00,0,74,1,Taiwán observa los contactos de China Popular ...,0,3
4,0,2017-10-07 15:10:01,0,10,0,La Policía admite que aún desconoce el “motivo...,0,1


In [159]:
date_news['month'] = date_news.dates.dt.month
date_news['year'] = date_news.dates.dt.year
two16 = date_news.loc[date_news.year == 2016]
date_news.groupby('year').count()

,angry,dates,haha,like,love,messages,sad,wow,month
year,,,,,,,,,
2009,147,147,147,147,147,147,147,147,147
2010,50,50,50,50,50,50,50,50,50
2011,141,141,141,141,141,141,141,141,141
2012,60,60,60,60,60,60,60,60,60
2013,299,299,299,299,299,299,299,299,299
2014,593,593,593,593,593,593,593,593,593
2015,304,304,304,304,304,304,304,304,304
2016,54,54,54,54,54,54,54,54,54
2017,797,797,797,797,797,797,797,797,797


In [182]:
import re
def get_url(url):
    #Extrae la URL de un string.
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', url)
    try:
        result =   urls[0]
    except:
        result = 'Not found'
    return result

valid_news['url'] = valid_news.messages.apply(get_url)
final_news = valid_news.loc[valid_news.url != 'Not found']
final_news.shape

C:\Users\juan9\Anaconda3Fix\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(2317, 9)

### El dataset final resultante es guardado en un csv.

In [185]:
final_news.to_csv('diario_libre_fb.csv',index = False)